---
title: Randomized Factorization-based Solvers
description: Fast linear regression using randomized QR factorization and half preconditioned normal equations
keywords: [randomized Cholesky QR, half preconditioned normal equations, linear regression, factorization-based solvers, sketched QR]
numbering:
  equations:
    enumerator: 4.%s
    continue: true
  proof:theorem:
    enumerator: 4.%s
    continue: true
  proof:algorithm:
    enumerator: 4.%s
    continue: true
  proof:definition:
    enumerator: 4.%s
    continue: true
  proof:proposition:
    enumerator: 4.%s
    continue: true
---

As noted in the introduction to this chapter, the classical direct approach to solving  the {eq}`task-regression` problem is to first compute a factorization (e.g. QR, SVD) of $\vec{A}$ in $O(nd^2)$ operations, and then use the factorization to solve the problem in $O(nd)$ operations.
The factorization step is typically the computational bottleneck, but the algorithms discussed in the previous chapter can be applied in a black-box manner. 


:::{prf:algorithm} Randomized Cholesky QR Regression
:label: rand-chol-qr

**Input:** $\vec{A}\in\R^{n\times d}$, $\vec{b}\in\R^n$, sketching dimension $k$

1. Get $\vec{Q},\vec{R} = \Call{Randomized-Cholesky-QR}(\vec{A},k)$
1. Solve $\vec{R}\vec{x} = \vec{Q}^\T\vec{b}$

**Output:** $\vec{x}$
:::



Recall, however, that the $\vec{Q}$ factor produced by the randomized Cholesky QR algorithm is compute as $\vec{Q} = \vec{Q}_1 \vec{R}_1^{-1}$, where $\vec{R}_1$, where $\vec{R}_1$ is the Cholesky factor of $\vec{Q}_1^\T\vec{Q}_1$, and $\vec{Q}_1$ is the approximate orthogonal basis produced by the [Sketched QR algorithm](../03-QR-Factorization/randomized-cholesky-qr.ipynb#sketched-qr).
Since we only need to compute $\vec{Q}^\T\vec{b}$, but not $\vec{Q}$ itself, we can avoid forming forming the product $\vec{Q}_1\vec{R}_1^{-1}$ explicitly.
This results in the following implementation:

:::{prf:algorithm} Randomized Half Preconditioned Normal Equations
:label: rand-HPNE

**Input:** $\vec{A}\in\R^{n\times d}$, $\vec{b}\in\R^n$, sketching dimension $k$

1. Get $\vec{Q}_1,\vec{R}_1 = \Call{Sketched-QR}(\vec{A},k)$
1. Form $\vec{X} = \vec{Q}_1^\T\vec{Q}_1$
1. Compute Choleksy factorization $\vec{R} = \Call{chol}(\vec{X})$
1. $\vec{R} = \vec{R}_2\vec{R}_1$
1. Solve $\vec{R}_2^\T\vec{R}\vec{x} = \vec{Q}_1^\T\vec{b}$

**Output:** $\vec{x}$
:::

The name *half preconditioned normal equations* is based on {cite:p}`ipsen_25`.

We can easily implement the algorithm in Numpy:

In [16]:
def randomized_HPNE(A,b,k,zeta,rng):

    Q1,R1 = sketched_qr(A,k,zeta,rng)
    X = Q1.T@Q1
    R2 = np.linalg.cholesky(X).T
    R = R2@R1
    y = sp.linalg.solve_triangular(R2.T,Q1.T@b,lower=True)
    x = sp.linalg.solve_triangular(R,y,lower=False)

    return x

## Numerical Example

Let's compare the performance of the three approaches described above.

In [17]:
import numpy as np
import scipy as sp
import time
import pandas as pd

import sys
sys.path.append('../')
from randnla import *

In [20]:
# Define randomized Cholesky QR regression algorithm
def randomized_cholesky_QR_regression(A,b,k,zeta,rng):

    Q,R = randomized_cholesky_QR(A,k,zeta,rng)
    x = sp.linalg.solve_triangular(R,Q.T@b,lower=False)    

    return x


# Generate a random matrix with controlled condition number
n = 5000
d = 300

U, s, Vt = np.linalg.svd(np.random.rand(n, d), full_matrices=False)
s = np.geomspace(1e-4, 1, d)  # Controlled singular values for numerical stability
A = U @ np.diag(s) @ Vt

b_norm = 1
residual_norm = .1

v = np.random.randn(n)
v_span = U @ (U.T @ v)
v_perp = v - v_span
v_span /= np.linalg.norm(v_span)
v_perp /= np.linalg.norm(v_perp)

b = v_span * np.sqrt(b_norm**2 - residual_norm**2) + v_perp * residual_norm

x_true = Vt.T@np.diag(1/s)@U.T@b
rng = np.random.default_rng(0) 

k = int(1.5*d)
zeta = 4


# Define QR factorization methods
qr_methods = {
    'Numpy': {
        'func': lambda: np.linalg.lstsq(A, b, rcond=None)[0],
    },
    'Randomized Cholesky QR': {
        'func': lambda: randomized_cholesky_QR_regression(A,b,k,zeta,rng),
    },
    'Randomized HPNE': {
        'func': lambda: randomized_HPNE(A,b,k,zeta,rng),
    },
}
# Time the QR factorization methods
n_repeat = 10  # Number of repetitions for averaging

results = []

for method_name, method_info in qr_methods.items():
    # Time the method
    start = time.time()
    for _ in range(n_repeat):
        x = method_info['func']()
    end = time.time()
    
    avg_time = (end - start) / n_repeat
    
    # Compute accuracy metrics
    results.append({
        'method': method_name,
        'time (s)': avg_time,
        'error': np.linalg.norm(x_true - x)/np.linalg.norm(x_true),
    })

# Create DataFrame and compute relative performance
results_df = pd.DataFrame(results)
results_df['speedup'] = results_df['time (s)'] / results_df['time (s)'].max()

# Display results with formatting
results_df.reindex(columns=['method','time (s)','speedup','error']).style.format({
    'time (s)': '{:.4f}',
    'speedup': '{:.1f}x',
    'error': '{:1.1e}',
})

,method,time (s),speedup,error
0,Numpy,0.0678,1.0x,4.3e-14
1,Randomized Cholesky QR,0.0520,0.8x,8.2e-14
2,Randomized HPNE,0.0421,0.6x,6.1e-14


The results are as expected. All methods get similar accuracy, but the randomized methods are faster due to offloading most of the computation to matrix-matrix multiplication. 
The optimized variant is slightly faster than the typical Randomized Cholesky QR-based method.